In [ ]:
import pandas as pd

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving events filters к тестовому Product Analyst Q1 2024.csv to events filters к тестовому Product Analyst Q1 2024 (3).csv


In [ ]:
events_filters = pd.read_csv('events filters к тестовому Product Analyst Q1 2024.csv')

In [ ]:
events_filters.head()

,event_category,event_action,event_label,total_events,unique_events
0,city_landing,price_button_submit,Tula / Показать предложения (11),8,7
1,city_landing,price_button_submit,Sochi / Показать предложения (39),12,10
2,city_landing,search-tools-button_open,Penza / Сортировка,2,2
3,city_landing,filters-categories_click,Gelendzhik / ЭКСКУРСИИ В ГЕЛЕНДЖИКЕ ЦЕНЫ,1,1
4,city_landing,price_button_submit,Moscow / Показать предложения (345),2,2


In [ ]:
events_filters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19092 entries, 0 to 19091
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   event_category  19092 non-null  object
 1   event_action    19092 non-null  object
 2   event_label     19092 non-null  object
 3   total_events    19092 non-null  int64 
 4   unique_events   19092 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 745.9+ KB


In [ ]:
# выделяем города из event_label

split_event_label = events_filters['event_label'].str.split('/', expand=True)
split_event_label.columns = ['event_city', 'event_label']

# чистим города
split_event_label['event_city'] = (split_event_label['event_city']
                    .str.strip()
                    .str.title()
                    .fillna("Unknown"))

#очистка event_label
split_event_label['event_label'] = (split_event_label['event_label']
                                    .str.lower()
                                    .str.replace(r'[0-9.,:;()!?\[\]{}<>+=\-*/«»—…]', '', regex=True)
                                    .str.strip()
                                    )

In [ ]:
# структурирование event_label для 'показать предложения' и 'not found'
offer = split_event_label['event_label'].str.contains(
    r'показать предложения|angebote anzeigen|показати пропозиції|show offers|הצג הצעות', regex=True, na=False
)

not_found = split_event_label['event_label'].str.contains(
    r'ничего не найдено|nothing found|nichts gefunden', regex=True, na=False
)

split_event_label.loc[offer, 'event_label'] = 'показать предложения'
split_event_label.loc[not_found, 'event_label'] = 'not found'

In [ ]:
df_events_filters = pd.concat([events_filters.drop(columns='event_label'), split_event_label], axis=1)
df_events_filters.head()

,event_category,event_action,total_events,unique_events,event_city,event_label
0,city_landing,price_button_submit,8,7,Tula,показать предложения
1,city_landing,price_button_submit,12,10,Sochi,показать предложения
2,city_landing,search-tools-button_open,2,2,Penza,сортировка
3,city_landing,filters-categories_click,1,1,Gelendzhik,экскурсии в геленджике цены
4,city_landing,price_button_submit,2,2,Moscow,показать предложения


# **1**.	сколько людей пользуются фильтрами?

In [ ]:
count_session_action = df_events_filters.groupby('event_action')[['unique_events', 'total_events']].sum().reset_index()
count_session_action

,event_action,unique_events,total_events
0,Page Visit,1384478,2124993
1,clear_filter_mobile,2788,3599
2,dates_filter_mobile,10272,13038
3,end_date_click,11738,14638
4,filters-categories_click,62539,80065
5,pay-type_checkbox,5465,7321
6,price_button_close,11646,14247
7,price_button_submit,63063,71890
8,price_first,6676,8572
9,price_second,1995,2826


In [ ]:
def get_events_count(action, column_name):  # функция для вывода отдельных значений из таблицы
  index_event_action = count_session_action.loc[count_session_action['event_action'] == action].index
  result = count_session_action[column_name][index_event_action].iloc[0]
  return result

# приближённое общее кол-во уникальных  пользователей -  Page Visit - т.к это просмотр страницы с этого начинается сессия
unique_session = get_events_count('Page Visit', 'unique_events')
print('Общее кол-во пользователей:', unique_session)

# приближённое кол-во пользователей, которые начали фильтрацию , но не обязательно применили ее - search-tools-button_open
unique_session_open_filter = get_events_count('search-tools-button_open', 'unique_events')
print('Кол-во пользователей, открывших фильтры:', unique_session_open_filter)

# приближённое количество уникальных пользователей, которые использовали фильтры до конца (нажали применить)
unique_session_applied_filter = get_events_count('price_button_submit', 'unique_events')
print('Кол-во пользователей, применивших фильтрацию:', unique_session_applied_filter)

Общее кол-во пользователей: 1384478
Кол-во пользователей, открывших фильтры: 85773
Кол-во пользователей, применивших фильтрацию: 63063


In [ ]:
СR1 = unique_session_open_filter/unique_session
print('Фильтры использовали примерно', round(СR1*100,2),'% от всех пользователей')

СR2 = unique_session_applied_filter/unique_session
print('Фильтры применили примерно', round(СR2*100,2),'% от всех пользователей')

CR3 = unique_session_applied_filter/unique_session_open_filter
print('Фильтры применили примерно', round(CR3*100),'% от пользователей, открывших фильтры')


Фильтры использовали примерно 6.2 % от всех пользователей
Фильтры применили примерно 4.56 % от всех пользователей
Фильтры применили примерно 74 % от пользователей, открывших фильтры


# **2.	в каких городах фильтрами пользуются больше? в каких меньше?**

In [ ]:
city_session = df_events_filters.loc[df_events_filters['event_action'] == 'Page Visit'].groupby('event_city')[['unique_events','total_events']].sum().reset_index()

In [ ]:
#выделены данные, где применен фильтр (price_button_submit)
event_action_applied_filter = df_events_filters.loc[df_events_filters['event_action'] == 'price_button_submit']
city_filter = event_action_applied_filter.groupby('event_city')[['unique_events','total_events']].sum().reset_index()

In [ ]:
city_filter_count = pd.merge(city_session , city_filter, how = 'left' , on = 'event_city').rename(columns={
                                      'unique_events_x': 'unique_events',
                                      'total_events_x': 'total_events',
                                      'unique_events_y': 'unique_events_applied_filter',
                                      'total_events_y': 'total_events_applied_filter'}).fillna(0)

In [ ]:
# расчет доли уникальных сессий , в которых использовался фильтр к общему кол-ву уникальных сессий
city_filter_count['filter_applied_rate'] = round(city_filter_count['unique_events_applied_filter'] / city_filter_count['unique_events'] * 100, 2)

In [ ]:
# использовала 80-й процентиль с целью сфокусироваться на основной массе данных, исключив выбросы (города где мало событий)

p = 0.80
top_min_unique_events = city_filter_count['unique_events'].quantile(p)
print(top_min_unique_events)

945.2000000000005


In [ ]:
city_filter_count.loc[city_filter_count['unique_events'] >= top_min_unique_events][
    ['event_city', 'unique_events', 'unique_events_applied_filter', 'filter_applied_rate']
    ].sort_values(by = 'filter_applied_rate', ascending=False).head(10)

,event_city,unique_events,unique_events_applied_filter,filter_applied_rate
67,Belgrade,1066,185.0,17.35
45,Ayia Napa,1881,293.0,15.58
631,Yerevan,6305,767.0,12.16
147,Cyprus,4197,494.0,11.77
99,Budapest,15770,1843.0,11.69
555,Tbilisi,9747,1081.0,11.09
292,Krakow,1237,134.0,10.83
629,Yaroslavl,4574,474.0,10.36
604,Vienna,2564,249.0,9.71
58,Barcelona,2218,202.0,9.11


In [ ]:
city_filter_count.loc[city_filter_count['unique_events'] >= top_min_unique_events][
    ['event_city', 'unique_events', 'unique_events_applied_filter', 'filter_applied_rate']
    ].sort_values(by = 'filter_applied_rate').head(10)

,event_city,unique_events,unique_events_applied_filter,filter_applied_rate
35,Arkhipo-Osipovka,1123,0.0,0.00
87,Borovoe,1214,0.0,0.00
162,Dombay,973,0.0,0.00
173,Dzhubga,2615,0.0,0.00
528,Sol-Iletsk,1830,0.0,0.00
633,Yeysk,1626,0.0,0.00
129,Chernobyl,3334,21.0,0.63
595,Varadero,994,7.0,0.70
39,Astana,1648,13.0,0.79
394,New Athos,2155,20.0,0.93


In [ ]:
city_max_count_session_filter = []

for city in city_filter_count['event_city']:
  city_index = city_filter_count.loc[city_filter_count['event_city'] == city].index
  if ((city_filter_count['unique_events'][city_index].iloc[0])> top_min_unique_events and
  (city_filter_count['filter_applied_rate'][city_index].iloc[0]) > 9):
    city_max_count_session_filter.append(city)
print('Cписок городов, где больше всего пользуются фильтрами:', city_max_count_session_filter)

city_min_count_session_filter = []

for city in city_filter_count['event_city']:
  city_index = city_filter_count.loc[city_filter_count['event_city'] == city].index
  if ((city_filter_count['unique_events'][city_index].iloc[0])> top_min_unique_events and
  (city_filter_count['filter_applied_rate'][city_index].iloc[0]) < 1):
    city_min_count_session_filter.append(city)
print('Cписок городов, где меньше всего пользуются фильтрами:', city_min_count_session_filter)

Cписок городов, где больше всего пользуются фильтрами: ['Ayia Napa', 'Barcelona', 'Belgrade', 'Budapest', 'Cyprus', 'Krakow', 'Tbilisi', 'Vienna', 'Yaroslavl', 'Yerevan']
Cписок городов, где меньше всего пользуются фильтрами: ['Abrau-Durso', 'Arkhipo-Osipovka', 'Astana', 'Borovoe', 'Chernobyl', 'Dombay', 'Dzhubga', 'New Athos', 'Sol-Iletsk', 'Varadero', 'Yeysk']


# 3.	какие разделы фильтров наиболее востребованы? “фильтры”, “сортировка”, “категории”?

In [ ]:
# классифицируем использование фильтра ( по разделам фильтрации : “фильтры”, “сортировка”, “категории”,
# исключаем event_action = no_filter - т.к данные действия не связаны с фильтрацией(не выбирают разделы фильтров) , filters-categories_click - только сортировка и категории

df_events_filters['event_action_type'] = None

# присваеваем "фильтры" action, которые относятся к разделу фильтры
no_filter = ['Page Visit','search-tools-button_open','price_button_submit',
             'price_button_close', 'clear_filter_mobile','filters-categories_click']
df_events_filters.loc[~df_events_filters['event_action'].isin(no_filter),
                      'event_action_type'] = 'фильтры'


# присваеваем "категория" для действий filters-categories_click , сначала по ключевым словам , чтобы сократить кол-во данных
category_1 = (
    (df_events_filters['event_action'] == 'filters-categories_click') &
    (df_events_filters['event_label'].str.contains(
        'экскурс|тур|гид|в |на |для ',
        regex=True, na=False))
    )
df_events_filters.loc[category_1,'event_action_type'] = 'категории'


# присваеваем "сортировка" для действий filters-categories_click , сначала по ключевым словам
sorting = (
    (df_events_filters['event_action'] == 'filters-categories_click') &
    (df_events_filters['event_label'].str.contains(
        'сначала|популярн|рейтинг|самый|rating|popular|cheap|הפופולרי ביותר ראשון',
        regex=True, na=False))
    )
df_events_filters.loc[sorting,'event_action_type'] = 'сортировка'


# присваеваем "категория" для оставшихся действий filters-categories_click
category_2 = (
   (df_events_filters['event_action'] == 'filters-categories_click') &
   (df_events_filters['event_action_type'].isna())
   )
df_events_filters.loc[category_2,'event_action_type'] = 'категории'

In [ ]:
df_event_action_type = df_events_filters.groupby('event_action_type').agg(
                                                   count_event_unique = ('unique_events', 'sum'),
                                                   count_event_total = ('total_events', 'sum')
                                                   ).reset_index()

df_event_action_type['rate_action_type_unique'] = round(df_event_action_type['count_event_unique'] /
                                                        df_event_action_type['count_event_unique'].sum() * 100, 2)
df_event_action_type['rate_action_type_total'] =  round(df_event_action_type['count_event_total'] /
                                                        df_event_action_type['count_event_total'].sum() * 100, 2)

In [ ]:
df_event_action_type

,event_action_type,count_event_unique,count_event_total,rate_action_type_unique,rate_action_type_total
0,категории,40621,53654,30.84,31.53
1,сортировка,21918,26411,16.64,15.52
2,фильтры,69160,90120,52.51,52.95


# 4.	как часто люди пользуются выбором цены?

In [ ]:
# расчет кол-во сессий , где использован фильтр - выбор цены
price_filter = ['price_first', 'price_second','price_third']
price_filter_total = 0
price_filter_unique = 0
for type_price in price_filter:
  price_filter_total += get_events_count(type_price, 'total_events') # функция для вывода отдельных значений из таблицы
  price_filter_unique += get_events_count(type_price, 'unique_events')

price_first = get_events_count('price_first', 'unique_events')
price_second = get_events_count('price_second', 'unique_events')
price_third = get_events_count('price_third', 'unique_events')

print('price_first', price_first , '\nprice_second', price_second,'\nprice_third', price_third)
print('\nrate_nprice_first', round(price_first / price_filter_total * 100, 2)  , '\nrate_price_second', round(price_second / price_filter_total * 100, 2),'\nrate_price_third', round(price_third / price_filter_total * 100, 2))

price_first 6676 
price_second 1995 
price_third 595

rate_nprice_first 54.27 
rate_price_second 16.22 
rate_price_third 4.84


In [ ]:
#1. Доля событий с фильтром
total_page_visits = get_events_count('Page Visit', 'total_events')
total_open_filter = get_events_count('search-tools-button_open', 'total_events')
total_applied_filter = get_events_count('price_button_submit', 'total_events')

print('Количество событий , где использован фильтр выбора цены',
      price_filter_total)
print('Доля событий с выбором цены',
      round(price_filter_total / total_page_visits * 100, 2),
      'от всех событий')
print('Доля событий с выбором цены',
      round(price_filter_total / total_open_filter * 100, 2),
      'от всех событий, открывших фильтры')
print('Доля событий с выбором цены',
      round(price_filter_total / total_applied_filter * 100, 2),
      'от всех событий, применивших фильтры')

Количество событий , где использован фильтр выбора цены 12302
Доля событий с выбором цены 0.58 от всех событий
Доля событий с выбором цены 13.06 от всех событий, открывших фильтры
Доля событий с выбором цены 17.11 от всех событий, применивших фильтры


In [ ]:
#2. Доля уникальных событий (сессий), использовавших фильтр
print('Количество уникальных сессий (приблизительное кол-во пользователей), где использован фильтр выбора цены',
      price_filter_unique)
print('Доля уникальных сессий с выбором цены',
      round(price_filter_unique / unique_session * 100, 2),
      'от всех уникальных сессий')
print('Доля уникальных сессий с выбором цены',
      round(price_filter_unique /  unique_session_open_filter * 100, 2),
      'от всех уникальных сессий, открывших фильтры')
print('Доля уникальных сессий с выбором цены',
      round(price_filter_unique / unique_session_applied_filter * 100, 2),
      'от всех уникальных сессий, применивших фильтры')

Количество уникальных сессий (приблизительное кол-во пользователей), где использован фильтр выбора цены 9266
Доля уникальных сессий с выбором цены 0.67 от всех уникальных сессий
Доля уникальных сессий с выбором цены 10.8 от всех уникальных сессий, открывших фильтры
Доля уникальных сессий с выбором цены 14.69 от всех уникальных сессий, применивших фильтры


In [ ]:
#3. Количество фильтров на пользователя

print(round(price_filter_total/price_filter_unique, 2))


1.33
